<a href="https://colab.research.google.com/github/moridin04/CCMACLRL_EXAM_COM221ML/blob/main/Midterm_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [89]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [91]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [92]:
df.dropna(inplace=True)

In [93]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [94]:
df.describe()

,id,model_year,milage,price
count,162610.000000,162610.000000,162610.000000,1.626100e+05
mean,94197.039137,2015.158822,71105.487086,4.081699e+04
std,54467.770815,5.699531,50157.874111,7.653857e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47026.250000,2012.000000,29000.000000,1.570000e+04
50%,94139.000000,2016.000000,64300.000000,2.850000e+04
75%,141421.750000,2019.000000,101000.000000,4.699900e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [95]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
137399,137399,Mitsubishi,Eclipse Spyder GT,2001,99500,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Gray,At least 1 accident or damage reported,Yes,11900
52802,52802,Nissan,Pathfinder Platinum,2018,112000,Gasoline,260.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,CVT Transmission,White,Beige,None reported,Yes,28300
45338,45338,Mercedes-Benz,AMG C 63 Base,2019,78500,Gasoline,503.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,9-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,68000
176194,176194,Mercedes-Benz,CLK-Class CLK 350,2007,161000,Gasoline,268.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,22500
125691,125691,INFINITI,QX80 Base,2017,90000,Gasoline,400.0HP 5.6L 8 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,24995
56127,56127,Porsche,911 GT3,2016,15000,Gasoline,475.0HP 3.8L Flat 6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes,210000
14544,14544,Lincoln,Navigator Reserve,2018,80000,Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Beige,At least 1 accident or damage reported,Yes,43000
99317,99317,Tesla,Model X Long Range,2021,250,Gasoline,670.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,69250
21378,21378,Jeep,Compass Sport,2010,136000,Gasoline,172.0HP 2.4L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Beige,At least 1 accident or damage reported,Yes,12000
48457,48457,Mercedes-Benz,SL-Class SL 550,2022,15183,Gasoline,429.0HP 4.7L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,63500


In [96]:
from datetime import datetime

current_year = datetime.now().year
df['car_age'] = current_year - df['model_year'].astype(int)
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car_age
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3


In [97]:
# Brand, Model, Fuel Type, Engine, Transmission, Ext_Col, Int_Col, Accident, Clean_Title
drop_col=['model_year']
df.drop(drop_col,inplace=True,axis=1)
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns]

,brand,model,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,MINI,Cooper S Base,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes
1,Lincoln,LS V8,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes
2,Chevrolet,Silverado 2500 LT,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes
3,Genesis,G90 5.0 Ultimate,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes
4,Mercedes-Benz,Metris Base,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes
...,...,...,...,...,...,...,...,...,...
188527,Chevrolet,Camaro Z28,Gasoline,310.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,White,Gray,None reported,Yes
188528,Cadillac,Escalade ESV Platinum,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes


In [98]:
df[object_columns]

,brand,model,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,MINI,Cooper S Base,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes
1,Lincoln,LS V8,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes
2,Chevrolet,Silverado 2500 LT,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes
3,Genesis,G90 5.0 Ultimate,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes
4,Mercedes-Benz,Metris Base,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes
...,...,...,...,...,...,...,...,...,...
188527,Chevrolet,Camaro Z28,Gasoline,310.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,White,Gray,None reported,Yes
188528,Cadillac,Escalade ESV Platinum,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes


In [115]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder = LabelEncoder()
df['brand_label'] = label_encoder.fit_transform(df['brand'])
df['model_label'] = label_encoder.fit_transform(df['model'])
df['fuel_type_label'] = label_encoder.fit_transform(df['fuel_type'])
df['engine_label'] = label_encoder.fit_transform(df['engine'])
df['transmission_label'] = label_encoder.fit_transform(df['transmission'])
df['ext_col_label'] = label_encoder.fit_transform(df['ext_col'])
df['int_col_label'] = label_encoder.fit_transform(df['int_col'])
df['accident_label'] = label_encoder.fit_transform(df['accident'])
df['clean_title_label'] = label_encoder.fit_transform(df['clean_title'])

In [116]:
df

,id,brand,model,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,car_age,brand_label,model_label,fuel_type_label,engine_label,transmission_label,ext_col_label,int_col_label,accident_label,clean_title_label
0,0,MINI,Cooper S Base,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,...,17,31,493,2,110,38,306,71,1,0
1,1,Lincoln,LS V8,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,...,22,28,925,2,358,38,257,10,0,0
2,2,Chevrolet,Silverado 2500 LT,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,...,22,9,1567,1,627,38,37,71,1,0
3,3,Genesis,G90 5.0 Ultimate,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,...,7,16,755,2,846,49,28,14,1,0
4,4,Mercedes-Benz,Metris Base,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,...,3,36,1072,2,252,23,28,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,Chevrolet,Camaro Z28,110000,Gasoline,310.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,White,Gray,None reported,...,25,9,402,2,604,38,298,71,1,0
188528,188528,Cadillac,Escalade ESV Platinum,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,...,7,8,601,2,849,49,298,10,1,0
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,...,6,36,204,2,757,31,298,14,0,0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,...,3,36,221,2,904,23,298,14,1,0


In [137]:
df1 = df.drop(['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title'],axis=1)
df1

,id,milage,price,car_age,brand_label,model_label,fuel_type_label,engine_label,transmission_label,ext_col_label,int_col_label,accident_label,clean_title_label
0,0,213000,4200,17,31,493,2,110,38,306,71,1,0
1,1,143250,4999,22,28,925,2,358,38,257,10,0,0
2,2,136731,13900,22,9,1567,1,627,38,37,71,1,0
3,3,19500,45000,7,16,755,2,846,49,28,14,1,0
4,4,7388,97500,3,36,1072,2,252,23,28,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,110000,14500,25,9,402,2,604,38,298,71,1,0
188528,188528,49000,27500,7,8,601,2,849,49,298,10,1,0
188529,188529,28600,30000,6,36,204,2,757,31,298,14,0,0
188530,188530,13650,86900,3,36,221,2,904,23,298,14,1,0


In [138]:
from sklearn.preprocessing import MinMaxScaler
non_boolean_numerical_features = ['milage' ,'price', 'car_age']
scaler = MinMaxScaler()
df1[non_boolean_numerical_features] = scaler.fit_transform(df1[non_boolean_numerical_features])

In [139]:
y_scaled = df1['price']
y_scaled

,price
0,0.000745
1,0.001016
2,0.004031
3,0.014566
4,0.032350
...,...
188527,0.004234
188528,0.008638
188529,0.009485
188530,0.028759


In [140]:
x = df1.drop(['price'],axis=1)
x

,id,milage,car_age,brand_label,model_label,fuel_type_label,engine_label,transmission_label,ext_col_label,int_col_label,accident_label,clean_title_label
0,0,0.525809,0.34,31,493,2,110,38,306,71,1,0
1,1,0.353544,0.44,28,925,2,358,38,257,10,0,0
2,2,0.337444,0.44,9,1567,1,627,38,37,71,1,0
3,3,0.047913,0.14,16,755,2,846,49,28,14,1,0
4,4,0.018000,0.06,36,1072,2,252,23,28,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,0.271425,0.50,9,402,2,604,38,298,71,1,0
188528,188528,0.120771,0.14,8,601,2,849,49,298,10,1,0
188529,188529,0.070388,0.12,36,204,2,757,31,298,14,0,0
188530,188530,0.033465,0.06,36,221,2,904,23,298,14,1,0


1. Use the Dataset file to train your model. Split this into train and test to calculate the RMSE

In [141]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y_scaled, test_size=0.2, random_state=42)

In [142]:
model = LinearRegression()

In [143]:
model.fit(X_train, y_train)


LinearRegression()

In [144]:
y_pred = model.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

0.02677687401997246

In [145]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt = pd.read_csv(test_url)

In [147]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [ ]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'class': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")